In [1]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import keras
import datetime as dt
import time

from keras import layers
from keras import models
from tensorflow.keras import optimizers

from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_recall_fscore_support
from tensorflow.keras import backend as K

from ABNmodules_multiclass import *
from sklearn.metrics import f1_score

tf.set_random_seed(1004)
random.seed(1004)

os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

In [2]:

def score_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())
    return sorted(unique_classes)

def one_hot_encoding(one_hot_vector,y, class2index):
    ind=class2index[y]
    one_hot_vector[ind]=1
    return one_hot_vector

# Search for multi-label subjects
def searching_overlap(input_directory,class2index, input_file_names):
    multiclasses=[]
    multisubjects=[]
    number = []
    for file in input_file_names:
        f=file
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    if len(tmp)>1:
                        one_hot_vector = [0]*(len(class2index))
                        for c in tmp:
                            one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                        multiclasses.append(one_hot_vector)
                        multisubjects.append(g)
                        number.append(len(tmp))
    return multisubjects, multiclasses, number

def block_feature(sequence_en, minimum_len): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)
        #print(start)
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

def exploratory_look(input_directory,file, class2index):
    classes = []
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                print(tmp, len(tmp))
    return tmp     

# Get classes of sorted file names
def get_labels(input_directory,file, class2index):
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                one_hot_vector = [0]*(len(class2index))
                for c in tmp:
                    one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                
    return one_hot_vector

def randextract_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len, x_mean_final, x_std_final):
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]
    for file in curr_file_indices:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        clip_file = block_feature(tmp_file, minimum_len)
        clip_file -= x_mean_final
        clip_file /= x_std_final
        mel_files.append(clip_file)
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    
    concat = list(zip(mel_files, classes))
    random.shuffle(concat)
    mel_files, classes = zip(*concat)
    return mel_files, classes

def train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
#     acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps): # loops over batches
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)

        batch_mels = np.asarray(batch_mels)
        batch_labels = np.asarray(np.squeeze(batch_labels))
        
        batch_labels = [batch_labels, batch_labels]


# return of train
# 0 = total loss (attention branch + perception branch)
# 1 = loss of attention pred
# 2 = loss of perception pred
# 3 = f1 of attention pred
# 4 = f1 of perception pred     


        train_tmp = model.train_on_batch(batch_mels, batch_labels)
        loss_ = train_tmp[0]/2
        f1_ = np.mean((train_tmp[3], train_tmp[4]))
        loss.append(loss_)
        f1.append(f1_)

    loss = np.mean(np.array(loss))
    f1 = np.mean(np.array(f1))
    return loss, f1

def train_edit(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
#     acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps): # loops over batches
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)

        batch_mels = np.asarray(batch_mels)
        batch_labels = np.asarray(np.squeeze(batch_labels))


        heatmap = CAM_conv1D(minimum_len, n_channels, batch_mels, batch_labels, out_len, get_conv_out)        
        heatmap = np.asarray(heatmap)
        
        batch_labels = [batch_labels, batch_labels]


# return of train
# 0 = total loss (attention branch + perception branch)
# 1 = loss of attention pred
# 2 = loss of perception pred
# 3 = f1 of attention pred
# 4 = f1 of perception pred     

        train_tmp = model.train_on_batch([batch_mels, heatmap], batch_labels)
        loss_ = train_tmp[0]/2
        f1_ = np.mean((train_tmp[3], train_tmp[4]))
        loss.append(loss_)
        f1.append(f1_)

    loss = np.mean(np.array(loss))
    f1 = np.mean(np.array(f1))
    return loss, f1

def test(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    total_acc = 0
    total_f1 = 0
    
    y_true = []
    y_pred=[]
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        heatmap_files=[]
        label = get_labels(input_directory, file, class2index)
        label = [label]
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            clip_file = clip_file.reshape(1,minimum_len,n_channels)
            mel_files.append(clip_file)    
        mel_files = np.asarray(mel_files)
        mel_files = mel_files.reshape(steps,minimum_len,n_channels)
        logit = model.predict(mel_files)
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
        pred = np.zeros(len(logit))
        for ii, label in enumerate(logit):
            if label >= 0.5:
                pred[ii] = 1
            else:
                pred[ii] = 0
        pred = pred.tolist()
        y_pred.append(pred)
        label = get_labels(input_directory,file,class2index)
        y_true.append(label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
    final_acc = total_acc / i
#     micro_metrics = precision_recall_fscore_support(y_true, y_pred, average='micro')
#     class_metrics = precision_recall_fscore_support(y_true, y_pred, average=None)
    f1_classes = f1_score(y_true, y_pred, average=None)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    return final_acc, f1_classes, f1_micro



def test_edit(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    y_true = []
    y_pred = []
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        heatmap_files=[]
        label = get_labels(input_directory, file, class2index)
        label = [label]
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            clip_file = clip_file.reshape(1,minimum_len,n_channels)
            heatmap = CAM_conv1D(minimum_len, n_channels, clip_file, label, out_len, get_conv_out)
            mel_files.append(clip_file)    
            heatmap_files.append(heatmap)
        mel_files = np.asarray(mel_files)
        heatmap_files = np.asarray(heatmap_files)
        heatmap_files = heatmap_files.reshape(steps, out_len,1)
        mel_files = mel_files.reshape(steps,minimum_len,n_channels)
        logit = model.predict([mel_files, heatmap_files])
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
        pred = np.zeros(len(logit))
        for ii, label in enumerate(logit):
            if label >= 0.5:
                pred[ii] = 1
            else:
                pred[ii] = 0
        pred = pred.tolist()
        y_pred.append(pred)
        label = get_labels(input_directory,file,class2index)
        y_true.append(label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
    final_acc = total_acc / i
#     micro_metrics = precision_recall_fscore_support(y_true, y_pred, average='micro')
#     class_metrics = precision_recall_fscore_support(y_true, y_pred, average=None)
    f1_classes = f1_score(y_true, y_pred, average=None)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    return final_acc, f1_classes, f1_micro



batch_size = 32#16#20#32#5#2#1#10#32
minimum_len = 2880
epochs = 300#370#100#1000
loss_function = 'binary_crossentropy' 
rootdir = '../'
date = dt.datetime.today().strftime("%Y%m%d")
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_name = 'Raw_data_20200424' 
mel_directory = os.path.join(rootdir, mel_name)
# results_directory = os.path.join(rootdir, 'results_'+date+'_0_IEEE_n=1')
# results_directory = os.path.join(rootdir, 'results_'+date+'_editting_CAM_multiclass')
results_directory = os.path.join(rootdir, 'results_'+date+'_ABN_multiclass_V4_ABN_edit')
if not os.path.isdir(input_directory):
    os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
    os.mkdir(mel_directory)
if not os.path.isdir(results_directory):
    os.mkdir(results_directory)
        
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)
input_file_names = sorted(input_files)

unique_classes = get_unique_classes(input_directory, input_files)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a

np.shape(input_file_names)

x_mean_all = []
x_std_all = []
for file in input_file_names:
    x = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    x_mean = [np.mean(x[:,0]), np.mean(x[:,1]), np.mean(x[:,2]), np.mean(x[:,3]), np.mean(x[:,4]), np.mean(x[:,5]),
             np.mean(x[:,6]), np.mean(x[:,7]), np.mean(x[:,8]), np.mean(x[:,9]), np.mean(x[:,10]), np.mean(x[:,11])]
    
    x_std = [np.std(x[:,0]), np.std(x[:,1]), np.std(x[:,2]), np.std(x[:,3]), np.std(x[:,4]), np.std(x[:,5]),
             np.std(x[:,6]), np.std(x[:,7]), np.std(x[:,8]), np.std(x[:,9]), np.std(x[:,10]), np.std(x[:,11])]
    x_mean_all.append(x_mean)
    x_std_all.append(x_std) # yjs corrected on 2020-05-25
x_mean_final = np.mean(x_mean_all, axis=0)
x_std_final = np.mean(x_std_all, axis=0)


data, data_test = train_test_split(input_file_names, test_size = 0.2, train_size = 0.8, shuffle=True, random_state=1004)
data_train, data_val = train_test_split(data, test_size = 0.05, train_size = 0.95, shuffle=True, random_state=1004)
print(np.shape(data_train), np.shape(data_val), np.shape(data_test))

main_input = Input(shape=(minimum_len,12), dtype='float32', name='main_input')

(5225,) (276,) (1376,)


In [3]:
data_train # 2355

['A2355.mat',
 'A3446.mat',
 'A1814.mat',
 'A3575.mat',
 'A6197.mat',
 'A6727.mat',
 'A3974.mat',
 'A3380.mat',
 'A1978.mat',
 'A2265.mat',
 'A3514.mat',
 'A4147.mat',
 'A3852.mat',
 'A0245.mat',
 'A1846.mat',
 'A5979.mat',
 'A1696.mat',
 'A3454.mat',
 'A5164.mat',
 'A6827.mat',
 'A6778.mat',
 'A5403.mat',
 'A5567.mat',
 'A5923.mat',
 'A0254.mat',
 'A4950.mat',
 'A3926.mat',
 'A4083.mat',
 'A0223.mat',
 'A6569.mat',
 'A5393.mat',
 'A6343.mat',
 'A0300.mat',
 'A3324.mat',
 'A5812.mat',
 'A6865.mat',
 'A6600.mat',
 'A6153.mat',
 'A5947.mat',
 'A0999.mat',
 'A0613.mat',
 'A5423.mat',
 'A5316.mat',
 'A2158.mat',
 'A3799.mat',
 'A0598.mat',
 'A2677.mat',
 'A5973.mat',
 'A2911.mat',
 'A5053.mat',
 'A1400.mat',
 'A1283.mat',
 'A3298.mat',
 'A2962.mat',
 'A3459.mat',
 'A5462.mat',
 'A5589.mat',
 'A1946.mat',
 'A3583.mat',
 'A3842.mat',
 'A5419.mat',
 'A1267.mat',
 'A2669.mat',
 'A6353.mat',
 'A4129.mat',
 'A1017.mat',
 'A6503.mat',
 'A2249.mat',
 'A0345.mat',
 'A0445.mat',
 'A6595.mat',
 'A329

In [4]:
data_val # 5949

['A5949.mat',
 'A0268.mat',
 'A0938.mat',
 'A0379.mat',
 'A6235.mat',
 'A5085.mat',
 'A0368.mat',
 'A4547.mat',
 'A6477.mat',
 'A3075.mat',
 'A1011.mat',
 'A0821.mat',
 'A6566.mat',
 'A0688.mat',
 'A3682.mat',
 'A5270.mat',
 'A6875.mat',
 'A0698.mat',
 'A1369.mat',
 'A1633.mat',
 'A3438.mat',
 'A6648.mat',
 'A0988.mat',
 'A2271.mat',
 'A4479.mat',
 'A0499.mat',
 'A5788.mat',
 'A5377.mat',
 'A4074.mat',
 'A3279.mat',
 'A0944.mat',
 'A1244.mat',
 'A4372.mat',
 'A3159.mat',
 'A5350.mat',
 'A1703.mat',
 'A5599.mat',
 'A5319.mat',
 'A3609.mat',
 'A0018.mat',
 'A4472.mat',
 'A0262.mat',
 'A3497.mat',
 'A4132.mat',
 'A2589.mat',
 'A0033.mat',
 'A6427.mat',
 'A3516.mat',
 'A3639.mat',
 'A5699.mat',
 'A6117.mat',
 'A2881.mat',
 'A1386.mat',
 'A0592.mat',
 'A5711.mat',
 'A4696.mat',
 'A3192.mat',
 'A2427.mat',
 'A5336.mat',
 'A1233.mat',
 'A3805.mat',
 'A1748.mat',
 'A0407.mat',
 'A4002.mat',
 'A1318.mat',
 'A3105.mat',
 'A3764.mat',
 'A4894.mat',
 'A5889.mat',
 'A0225.mat',
 'A3407.mat',
 'A234

In [5]:
data_test # 0349

['A0349.mat',
 'A3947.mat',
 'A3900.mat',
 'A0817.mat',
 'A2140.mat',
 'A6793.mat',
 'A1879.mat',
 'A0900.mat',
 'A0454.mat',
 'A6855.mat',
 'A1142.mat',
 'A2316.mat',
 'A1701.mat',
 'A0892.mat',
 'A4156.mat',
 'A6654.mat',
 'A1902.mat',
 'A3887.mat',
 'A4328.mat',
 'A6705.mat',
 'A2393.mat',
 'A1654.mat',
 'A4534.mat',
 'A4223.mat',
 'A6232.mat',
 'A5215.mat',
 'A6361.mat',
 'A2060.mat',
 'A5460.mat',
 'A4804.mat',
 'A5066.mat',
 'A6283.mat',
 'A3889.mat',
 'A0621.mat',
 'A5856.mat',
 'A0943.mat',
 'A1936.mat',
 'A3808.mat',
 'A5473.mat',
 'A6224.mat',
 'A6848.mat',
 'A1420.mat',
 'A3335.mat',
 'A3258.mat',
 'A1623.mat',
 'A4062.mat',
 'A5721.mat',
 'A5733.mat',
 'A1238.mat',
 'A4654.mat',
 'A4504.mat',
 'A0987.mat',
 'A2839.mat',
 'A4243.mat',
 'A5778.mat',
 'A6173.mat',
 'A1782.mat',
 'A5404.mat',
 'A1443.mat',
 'A3470.mat',
 'A6876.mat',
 'A1230.mat',
 'A1870.mat',
 'A1186.mat',
 'A0736.mat',
 'A4685.mat',
 'A4562.mat',
 'A4444.mat',
 'A1568.mat',
 'A3403.mat',
 'A6465.mat',
 'A039

### Checking how many multilabel in each dataset

In [6]:
multi_train, _, _ = searching_overlap(input_directory, class2index, data_train)
multi_val, _, _ = searching_overlap(input_directory, class2index, data_val)
multi_test, _, _ = searching_overlap(input_directory, class2index, data_test)
print(len(multi_train))
print(len(multi_val))
print(len(multi_test))

366
19
91


### For attention editting using CAM extracted from primitive ABN

In [7]:
CAMdir = '/home/taejoon/PhysioNetChallenge/results_20200622_ABN_multiclass_V4_primitiveABN'
p_model = primitive_ABN((None, 12), 9, minimum_len, out_ch=256, n=1)
latest = tf.train.latest_checkpoint(CAMdir)
p_model.load_weights(latest)

conv_layer = 'activation_5'
softmax_layer = 'perception_branch_dense_2'

n_channels=12
out_len=12
get_conv_out = K.function(p_model.input, [p_model.get_layer(conv_layer).output, p_model.get_layer(softmax_layer).weights[0]])

def CAM_conv1D(minimum_len, n_channels, x, y, out_len, get_conv_out):
    
    # x랑 y는 batch size만큼의 리스트 (32)
    heatmaps=[]    
    
    curr_x = np.asarray(x)
    curr_x = curr_x.reshape(len(x),minimum_len,n_channels)
    
    conv_out, softmax_weights = get_conv_out(curr_x)

    for i in range(len(x)):

        curr_classes = y[i]
        class_index=[]
        [class_index.append(j) for j in range(len(curr_classes)) if curr_classes[j]==1]
        heatmap = np.zeros((1,12)) #heatmap=np.zeros((1,36)) # might need to fix this if GradCAM or primitive model changes

        conv_out_ = conv_out[i] # (36, 128) / (32, 12, 256)
#         print(conv_out.shape)

        for label in class_index:  # multiclass일 경우 대비해서 for문
            curr_weights = softmax_weights[:,label]
            weighted_conv = conv_out_*curr_weights
            
            weighted_conv = weighted_conv.sum(axis=-1) # output = (1,36)
            heatmap += weighted_conv
            
        heatmap %= len(class_index) # 단일 class일 경우 1로 나눠짐. 두개일 경우 더해진 heatmap들이 2로 나눠짐
#         heatmap = np.resize(heatmap, (1,out_len))
        heatmap = np.resize(heatmap, (out_len, 1))
        heatmaps.append(heatmap)
        
    return heatmaps
p_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, 12)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 64)     2368        input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, 64)     256         conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 64)     12352       batch_norm

In [8]:
# model = primitive_ABN((None,12), len(unique_classes), minimum_len)
# model = ABN_model((None,12), len(unique_classes), minimum_len)
model = edit_ABN_model((None,12), len(unique_classes), minimum_len)

model.summary()
model.compile(loss=loss_function,
              optimizer=optimizers.Adam(lr=1e-5),           
              metrics=[score_f1])



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, 12)]   0                                            
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (None, None, 64)     2368        input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, None, 64)     256         conv1d_21[0][0]                  
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, None, 64)     12352       batch_normalization_21[0][0]     
____________________________________________________________________________________________

In [9]:
class2index

{'AF': 0,
 'I-AVB': 1,
 'LBBB': 2,
 'Normal': 3,
 'PAC': 4,
 'PVC': 5,
 'RBBB': 6,
 'STD': 7,
 'STE': 8}

In [10]:
# # 중단된 training 이어돌리기위해 임시로 사용
# results_directory = results_directory.replace("0608", "0604") # 날짜 달라졌을때
# latest = tf.train.latest_checkpoint(results_directory)
# latest
# model.load_weights(latest)

In [11]:
# DISP DATETIME FOR CHECKING TIME
print(dt.datetime.now())

train_loss_sum=[]
train_f1_sum=[]
val_acc_sum=[]
val_f1_sum=[]
val_f1_classes=[]
val_f1_min = 0
print(results_directory)

for num_epoch in range(epochs):
#     num_epoch += 32 # 중단된 코드 돌리기 위해 임의로 사용
    random.shuffle(data_train)
#     train_loss, train_f1 = train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final)
    train_loss, train_f1 = train_edit(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final)
    train_loss_sum.append(train_loss)
    train_f1_sum.append(train_f1)
    
    print('\nEpoch',num_epoch,'train_loss:',f'{train_loss:.3f}','train_f1:',f'{train_f1:.3f}',"\t")
    model_output = "ECG_ABN_E%02dL%.2f" % (num_epoch, train_loss)
    save_name = os.path.join(results_directory, model_output)
    
#     val_acc, f1_classes, f1 = test(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)
    val_acc, f1_classes, f1 = test_edit(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)
    
    val_acc_sum.append(val_acc)
    val_f1_sum.append(f1)
    val_f1_classes.append(f1_classes)

    if f1 > val_f1_min:
        val_f1_min = f1
        if val_acc > 0.6: # 너무 낮아서 어차피 안쓸것들은 제외하기 위한 것
            model.save_weights(save_name.format(epoch=0))
    
    print('\nValidation', num_epoch, 'valid_f1:',f'{f1:.3f}', 'best_f1:',f'{val_f1_min:.3f}', 'mean accuracy:' f'{val_acc:.3f}',"\t")
#     print('\nValidation mean accuracy: ', f'{val_acc:.3f}', "\t")
#     print('\nValidation mean f1 by classes: ',f1_classes, "\t")

np.save(os.path.join(results_directory, 'train_loss_sum'), train_loss_sum)
np.save(os.path.join(results_directory, 'train_f1_sum'), train_f1_sum)
np.save(os.path.join(results_directory, 'val_acc_sum'), val_acc_sum)
np.save(os.path.join(results_directory, 'val_f1_sum'), val_f1_sum)
np.save(os.path.join(results_directory, 'val_f1_classes'), val_f1_classes)
print(dt.datetime.now())

2020-06-22 07:39:45.148958
../results_20200622_ABN_multiclass_V4_ABN_edit

Epoch 0 train_loss: 0.551 train_f1: 0.254 	

Validation 0 valid_f1: 0.321 best_f1: 0.321 mean accuracy:0.175 	

Epoch 1 train_loss: 0.451 train_f1: 0.335 	

Validation 1 valid_f1: 0.403 best_f1: 0.403 mean accuracy:0.240 	

Epoch 2 train_loss: 0.423 train_f1: 0.373 	

Validation 2 valid_f1: 0.483 best_f1: 0.483 mean accuracy:0.309 	

Epoch 3 train_loss: 0.403 train_f1: 0.412 	

Validation 3 valid_f1: 0.506 best_f1: 0.506 mean accuracy:0.335 	

Epoch 4 train_loss: 0.390 train_f1: 0.430 	

Validation 4 valid_f1: 0.559 best_f1: 0.559 mean accuracy:0.385 	

Epoch 5 train_loss: 0.379 train_f1: 0.447 	

Validation 5 valid_f1: 0.595 best_f1: 0.595 mean accuracy:0.418 	

Epoch 6 train_loss: 0.369 train_f1: 0.466 	

Validation 6 valid_f1: 0.628 best_f1: 0.628 mean accuracy:0.458 	

Epoch 7 train_loss: 0.361 train_f1: 0.481 	

Validation 7 valid_f1: 0.646 best_f1: 0.646 mean accuracy:0.469 	

Epoch 8 train_loss: 0.354 tra


Validation 71 valid_f1: 0.765 best_f1: 0.772 mean accuracy:0.655 	

Epoch 72 train_loss: 0.204 train_f1: 0.672 	

Validation 72 valid_f1: 0.758 best_f1: 0.772 mean accuracy:0.669 	

Epoch 73 train_loss: 0.203 train_f1: 0.674 	

Validation 73 valid_f1: 0.767 best_f1: 0.772 mean accuracy:0.658 	

Epoch 74 train_loss: 0.202 train_f1: 0.676 	

Validation 74 valid_f1: 0.758 best_f1: 0.772 mean accuracy:0.655 	

Epoch 75 train_loss: 0.200 train_f1: 0.678 	

Validation 75 valid_f1: 0.752 best_f1: 0.772 mean accuracy:0.640 	

Epoch 76 train_loss: 0.198 train_f1: 0.680 	

Validation 76 valid_f1: 0.771 best_f1: 0.772 mean accuracy:0.665 	

Epoch 77 train_loss: 0.197 train_f1: 0.680 	

Validation 77 valid_f1: 0.767 best_f1: 0.772 mean accuracy:0.673 	

Epoch 78 train_loss: 0.195 train_f1: 0.684 	

Validation 78 valid_f1: 0.754 best_f1: 0.772 mean accuracy:0.658 	

Epoch 79 train_loss: 0.193 train_f1: 0.686 	

Validation 79 valid_f1: 0.777 best_f1: 0.777 mean accuracy:0.669 	

Epoch 80 train_loss


Epoch 143 train_loss: 0.095 train_f1: 0.853 	

Validation 143 valid_f1: 0.770 best_f1: 0.780 mean accuracy:0.673 	

Epoch 144 train_loss: 0.096 train_f1: 0.852 	

Validation 144 valid_f1: 0.768 best_f1: 0.780 mean accuracy:0.680 	

Epoch 145 train_loss: 0.094 train_f1: 0.858 	

Validation 145 valid_f1: 0.773 best_f1: 0.780 mean accuracy:0.676 	

Epoch 146 train_loss: 0.092 train_f1: 0.860 	

Validation 146 valid_f1: 0.756 best_f1: 0.780 mean accuracy:0.658 	

Epoch 147 train_loss: 0.092 train_f1: 0.858 	

Validation 147 valid_f1: 0.773 best_f1: 0.780 mean accuracy:0.687 	

Epoch 148 train_loss: 0.089 train_f1: 0.867 	

Validation 148 valid_f1: 0.760 best_f1: 0.780 mean accuracy:0.658 	

Epoch 149 train_loss: 0.087 train_f1: 0.872 	

Validation 149 valid_f1: 0.782 best_f1: 0.782 mean accuracy:0.695 	

Epoch 150 train_loss: 0.088 train_f1: 0.871 	

Validation 150 valid_f1: 0.772 best_f1: 0.782 mean accuracy:0.680 	

Epoch 151 train_loss: 0.086 train_f1: 0.874 	

Validation 151 valid_f1:


Epoch 214 train_loss: 0.040 train_f1: 0.942 	

Validation 214 valid_f1: 0.776 best_f1: 0.796 mean accuracy:0.705 	

Epoch 215 train_loss: 0.041 train_f1: 0.940 	

Validation 215 valid_f1: 0.789 best_f1: 0.796 mean accuracy:0.713 	

Epoch 216 train_loss: 0.042 train_f1: 0.936 	

Validation 216 valid_f1: 0.763 best_f1: 0.796 mean accuracy:0.687 	

Epoch 217 train_loss: 0.041 train_f1: 0.942 	

Validation 217 valid_f1: 0.790 best_f1: 0.796 mean accuracy:0.720 	

Epoch 218 train_loss: 0.041 train_f1: 0.940 	

Validation 218 valid_f1: 0.777 best_f1: 0.796 mean accuracy:0.713 	

Epoch 219 train_loss: 0.040 train_f1: 0.943 	

Validation 219 valid_f1: 0.767 best_f1: 0.796 mean accuracy:0.684 	

Epoch 220 train_loss: 0.039 train_f1: 0.943 	

Validation 220 valid_f1: 0.757 best_f1: 0.796 mean accuracy:0.665 	

Epoch 221 train_loss: 0.039 train_f1: 0.942 	

Validation 221 valid_f1: 0.769 best_f1: 0.796 mean accuracy:0.680 	

Epoch 222 train_loss: 0.039 train_f1: 0.943 	

Validation 222 valid_f1:


Epoch 285 train_loss: 0.024 train_f1: 0.966 	

Validation 285 valid_f1: 0.760 best_f1: 0.796 mean accuracy:0.698 	

Epoch 286 train_loss: 0.023 train_f1: 0.968 	

Validation 286 valid_f1: 0.761 best_f1: 0.796 mean accuracy:0.684 	

Epoch 287 train_loss: 0.024 train_f1: 0.968 	

Validation 287 valid_f1: 0.773 best_f1: 0.796 mean accuracy:0.702 	

Epoch 288 train_loss: 0.024 train_f1: 0.966 	

Validation 288 valid_f1: 0.765 best_f1: 0.796 mean accuracy:0.705 	

Epoch 289 train_loss: 0.025 train_f1: 0.962 	

Validation 289 valid_f1: 0.752 best_f1: 0.796 mean accuracy:0.669 	

Epoch 290 train_loss: 0.024 train_f1: 0.965 	

Validation 290 valid_f1: 0.757 best_f1: 0.796 mean accuracy:0.665 	

Epoch 291 train_loss: 0.023 train_f1: 0.969 	

Validation 291 valid_f1: 0.777 best_f1: 0.796 mean accuracy:0.702 	

Epoch 292 train_loss: 0.024 train_f1: 0.966 	

Validation 292 valid_f1: 0.786 best_f1: 0.796 mean accuracy:0.720 	

Epoch 293 train_loss: 0.023 train_f1: 0.967 	

Validation 293 valid_f1:

### Now, test

In [22]:
def test_edit_final(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final, p_model):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    y_true = []
    y_pred=[]
    
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        mel_files_logit = []
        heatmap_files=[]

        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            clip_file = clip_file.reshape(1,minimum_len,n_channels)
            
            # 여기서 logit으로 y구함#####################
            logit_ = p_model.predict(clip_file)
            logit_ = np.mean(logit_, axis=0)
            logit_ = np.mean(logit_, axis=0)
            pred_ = np.zeros(len(logit_))
            for ii, label in enumerate(logit_):
                if label >= 0.5: 
                    pred_[ii] = 1
                else:
                    pred_[ii] = 0
            pred_ = pred_.tolist()
            ##########################################
            
            heatmap = CAM_conv1D(minimum_len, n_channels, clip_file, [pred_], out_len, get_conv_out)
            mel_files.append(clip_file)    
            heatmap_files.append(heatmap)
        mel_files = np.asarray(mel_files)
        heatmap_files = np.asarray(heatmap_files)
        heatmap_files = heatmap_files.reshape(steps, out_len,1)
        mel_files = mel_files.reshape(steps,minimum_len,n_channels)
        logit = model.predict([mel_files, heatmap_files])
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
        pred = np.zeros(len(logit))
        for ii, label in enumerate(logit):
            if label >= 0.5:
                pred[ii] = 1
            else:
                pred[ii] = 0
        pred = pred.tolist()
        y_pred.append(pred)
        label = get_labels(input_directory,file,class2index)
        y_true.append(label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
    final_acc = total_acc / i
    f1_classes = f1_score(y_true, y_pred, average=None)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    return final_acc, f1_classes, f1_micro

### test_edit_final2는 CAM에 label global하게 할당

In [34]:
def test_edit_final2(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final, p_model):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    mel_files = []
    classes = []
    
    y_true = []
    y_pred=[]    
    
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        mel_files_logit = []
        heatmap_files=[]

        for block in range(steps):
            # 여기서 logit으로 y구함
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            clip_file = clip_file.reshape(1,minimum_len,n_channels)
            mel_files_logit.append(clip_file)
            
        mel_files_logit = np.asarray(mel_files_logit)
        mel_files_logit = mel_files_logit.reshape(steps,minimum_len,n_channels)
        
        
        
        logit_ = p_model.predict(mel_files_logit)
        
        logit_ = np.mean(logit_, axis=0)
        logit_ = np.mean(logit_, axis=0)
#         print(logit_)
        pred_ = np.zeros(len(logit_))
        for ii, label in enumerate(logit_):
            if label >= 0.5: 
                pred_[ii] = 1
            else:
                pred_[ii] = 0
        
        pred_ = pred_.tolist()

#         print(pred_)
            #######
        
        
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            clip_file = clip_file.reshape(1,minimum_len,n_channels)

            heatmap = CAM_conv1D(minimum_len, n_channels, clip_file, [pred_], out_len, get_conv_out)
#             print(heatmap)
            mel_files.append(clip_file)    
            heatmap_files.append(heatmap)
        mel_files = np.asarray(mel_files)
        heatmap_files = np.asarray(heatmap_files)
        heatmap_files = heatmap_files.reshape(steps, out_len,1)
        mel_files = mel_files.reshape(steps,minimum_len,n_channels)
        logit = model.predict([mel_files, heatmap_files])
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
        pred = np.zeros(len(logit))
       
        print(logit)
        for ii, label in enumerate(logit):
            if label >= 0.5:
                pred[ii] = 1
            else:
                pred[ii] = 0
        pred = pred.tolist()
        y_pred.append(pred)
        print(pred)
        label = get_labels(input_directory,file,class2index)
        y_true.append(label)
        print(label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
    final_acc = total_acc / i
    f1_classes = f1_score(y_true, y_pred, average=None)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    return final_acc, f1_classes, f1_micro

In [12]:
# test_model = primitive_ABN((None,12), len(unique_classes), minimum_len)
# test_model = ABN_model((None,12), len(unique_classes), minimum_len)
test_model = edit_ABN_model((None,12), len(unique_classes), minimum_len)

latest_test = tf.train.latest_checkpoint(results_directory)
test_model.load_weights(latest_test)

test_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, 12)]   0                                            
__________________________________________________________________________________________________
conv1d_42 (Conv1D)              (None, None, 64)     2368        input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, None, 64)     256         conv1d_42[0][0]                  
__________________________________________________________________________________________________
conv1d_43 (Conv1D)              (None, None, 64)     12352       batch_normalization_42[0][0]     
____________________________________________________________________________________________

In [23]:
# test_acc, f1_classes, f1 = test(data_test, mel_directory, input_directory, class2index, minimum_len, test_model, x_mean_final, x_std_final)
test_acc, test_f1, test_summary = test_edit_final(data_test, mel_directory, input_directory, class2index, minimum_len, test_model, x_mean_final, x_std_final, p_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in remainder


In [24]:
test_acc

0.7156363636363636

In [25]:
test_f1

array([0.89236791, 0.86769231, 0.85106383, 0.7651715 , 0.31515152,
       0.59728507, 0.93055556, 0.71380471, 0.44827586])

In [26]:
test_summary

0.7942238267148014

In [35]:
test_acc, test_f1, test_summary = test_edit_final2(data_test, mel_directory, input_directory, class2index, minimum_len, test_model, x_mean_final, x_std_final, p_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in remainder


[0.668864   0.3045532  0.26661003 0.25836742 0.31192195 0.29303962
 0.25284642 0.34314018 0.39873388]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.7410046  0.05741327 0.0203369  0.00782228 0.07422818 0.22294042
 0.00750254 0.09726885 0.153659  ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0.7880262  0.05814265 0.02071559 0.00771914 0.06798032 0.36252668
 0.00817072 0.09444116 0.15420996]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.53461564 0.05915292 0.0219392  0.00779341 0.10127711 0.04541014
 0.00558129 0.09399413 0.1551045 ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.7405381  0.05778167 0.02031882 0.0080004  0.07194299 0.22188738
 0.00729612 0.09546929 0.15415958]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.77138805 0.05716988 0.02035418 0.00756945 0.07218704 0.27816635
 0.00807302 0.09505913 0.15346836]
[1.0, 0.0, 0.0, 0.

[0.7730883  0.05812912 0.02097407 0.00750749 0.07232127 0.27630544
 0.0082525  0.09522237 0.15356055]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.7268871  0.05886346 0.02111046 0.00737225 0.07174878 0.21068512
 0.00749227 0.09447503 0.15307716]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0.7721367  0.05818561 0.02136048 0.00759946 0.07146002 0.30831406
 0.00779719 0.09572314 0.1543742 ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.5260334  0.05855931 0.02086322 0.00768833 0.0988016  0.04422337
 0.00517778 0.09417446 0.15351453]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0.7440478  0.05093138 0.01684761 0.00661629 0.06294143 0.5370611
 0.00456136 0.0973365  0.14996178]
[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0.77985454 0.05847059 0.02156578 0.00758308 0.0710723  0.2991922
 0.00770517 0.09462136 0.15461153]
[1.0, 0.0, 0.0, 0.0,

[0.62898964 0.05962617 0.02110998 0.00798851 0.0803143  0.10958106
 0.00713703 0.09452314 0.15428951]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0.50627077 0.05741844 0.02025664 0.00800024 0.10556392 0.04382483
 0.00519225 0.0958043  0.15419923]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.4922331  0.05805363 0.0198089  0.00826423 0.10635497 0.04358849
 0.00531751 0.09648512 0.15362908]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0.79462945 0.05750498 0.02134146 0.00738364 0.06880669 0.34798658
 0.00821434 0.09417    0.15439418]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.71223855 0.05706897 0.0202297  0.00787207 0.07755825 0.14253472
 0.00706023 0.09703188 0.15378442]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0.76983416 0.05795867 0.02081456 0.00779301 0.0718902  0.2783475
 0.00772857 0.09602694 0.15418847]
[1.0, 0.0, 0.0, 0.0

[0.608236   0.05920864 0.02124995 0.0071524  0.07976621 0.05719248
 0.00619196 0.09031117 0.15312509]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0.7899838  0.05816279 0.02113757 0.00725217 0.06407771 0.52132475
 0.01036158 0.09647991 0.1537433 ]
[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.51263815 0.0581245  0.02066731 0.00798316 0.10402377 0.04313172
 0.00512165 0.09436193 0.1544945 ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.67061186 0.3038075  0.2665642  0.25836846 0.31285226 0.2932843
 0.25280678 0.3439973  0.39886135]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.76458097 0.05734769 0.02138507 0.00645901 0.07187738 0.13981047
 0.00624751 0.08837232 0.15215604]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0.5108082  0.05764385 0.02013174 0.00795801 0.10489137 0.0442376
 0.00522128 0.09623726 0.15361148]
[1.0, 0.0, 0.0, 0.0,

...

[0.72998893 0.05939303 0.02140536 0.00708397 0.06313001 0.54294765
 0.00788566 0.09885566 0.15272836]
[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.55894697 0.05902681 0.02111034 0.00848744 0.10201915 0.05293294
 0.00602823 0.10207748 0.15445358]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.736284   0.05813653 0.02079366 0.00779095 0.0732512  0.20310141
 0.00766629 0.09499401 0.15406086]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 1, 0]
[0.66797173 0.30559778 0.26712486 0.2588225  0.3141613  0.29511693
 0.2531079  0.3462736  0.3994346 ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.5101909  0.05842726 0.02066561 0.00795373 0.10586934 0.04413173
 0.00523903 0.09523068 0.15406226]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0.5472794  0.05781282 0.02069566 0.00752118 0.09372848 0.0437983
 0.00515524 0.09292356 0.15340795]
[1.0, 0.0, 0.0, 0.0

[0.76454026 0.0581416  0.02113304 0.00758349 0.07172506 0.25362343
 0.00739948 0.09489597 0.15405574]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.6632308  0.05872482 0.02189676 0.00731872 0.0806154  0.07686023
 0.00614616 0.09263162 0.15419891]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.49963546 0.05791284 0.0201077  0.00840612 0.10404043 0.04380636
 0.00527357 0.09667926 0.15429382]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0.5061202  0.05804586 0.0203637  0.00800616 0.10566075 0.04397292
 0.00519689 0.09598869 0.15400952]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0.502022   0.05784933 0.0200328  0.00839124 0.10709117 0.04447468
 0.00539335 0.09771629 0.15429273]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0.7562783  0.05791107 0.02039231 0.00820224 0.07331936 0.25215578
 0.00757435 0.0980158  0.15429434]
[1.0, 0.0, 0.0, 0.

[0.6750493  0.05993572 0.02181491 0.00743118 0.07659607 0.12064235
 0.00693992 0.09304422 0.15387718]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0.60670215 0.05829453 0.02135964 0.0075529  0.08764993 0.05615783
 0.00546946 0.09513482 0.15367939]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.58507156 0.05711833 0.02052952 0.00728352 0.09223379 0.04639128
 0.00511111 0.09303424 0.15274855]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0.7783116  0.05887631 0.02136736 0.00757557 0.07007457 0.2673505
 0.00795251 0.09345689 0.15408222]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.66860247 0.30505988 0.26693594 0.25842366 0.31238845 0.29362002
 0.25290164 0.34355903 0.398937  ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0.6691866  0.30436864 0.26654828 0.25835508 0.31234393 0.29328737
 0.2528466  0.34374937 0.3986187 ]
[1.0, 0.0, 0.0, 0.0

[0.7177358  0.05789933 0.02040594 0.00795135 0.07615711 0.1708988
 0.00727925 0.09784275 0.15374462]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0.54678524 0.06027974 0.02073123 0.00818467 0.0913097  0.04808098
 0.00590754 0.09491365 0.1538491 ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0.7280464  0.05887172 0.0209461  0.00748491 0.07054476 0.23476575
 0.00786673 0.09485668 0.1531508 ]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0.66973025 0.30457228 0.2669051  0.2584567  0.3127649  0.2933426
 0.25288776 0.34374595 0.39924487]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0.6630708  0.05706349 0.02018706 0.0078316  0.07784276 0.10975558
 0.00644674 0.09700849 0.15380317]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0.7637586  0.0582125  0.02101251 0.00696105 0.069323   0.17246987
 0.00670677 0.0904935  0.15249826]
[1.0, 0.0, 0.0, 0.0,

[0.5090929  0.05800066 0.02049399 0.00792769 0.1058996  0.04445673
 0.00519709 0.09588018 0.15405558]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0.68755144 0.06060421 0.02176264 0.00797999 0.07396579 0.18597439
 0.00774036 0.09626777 0.15424335]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[0.5980067  0.05740868 0.02069023 0.00735104 0.08800207 0.04594347
 0.00531229 0.09152339 0.15298662]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0.5984     0.05701029 0.01993746 0.00813808 0.09182094 0.05285111
 0.00592486 0.09703958 0.15356342]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 1, 0, 0, 1, 0]
[0.66611826 0.30410156 0.26616156 0.25895813 0.31395188 0.2939069
 0.25310382 0.34634647 0.39944574]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 1, 0]
[0.52608585 0.05850475 0.02069107 0.00785191 0.09862179 0.04382457
 0.00533897 0.093964   0.15408918]
[1.0, 0.0, 0.0, 0.0

In [36]:
test_acc

0.152

In [30]:
test_f1

array([0.89236791, 0.86769231, 0.85106383, 0.7651715 , 0.31515152,
       0.59728507, 0.93055556, 0.71380471, 0.44827586])

In [31]:
test_summary

0.7942238267148014

In [14]:
test_acc

0.7192727272727273

In [15]:
f1

0.7615658362989324

In [16]:
f1_classes

array([0.92156863, 0.82142857, 0.96296296, 0.56      , 0.33333333,
       0.60465116, 0.91729323, 0.75362319, 0.44444444])